<a href="https://colab.research.google.com/github/shubham6907/Python-Codes/blob/main/URM_Authorization_for_API_calls_using_bearer_token.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **URM Authorization for API calls using bearer token**

### **Contents of API function**

In [ ]:
@app.route("/")
def api_call():
        access_token = request.headers.get(constants.HEADER_AUTHORIZATION)
        content_type = request.headers.get(constants.CONTENT_TYPE_HEADER)

        token_validation_status, token_status = urm_vt.validate_token(constants.REQUEST_METHOD,constants.VALIDATE_TOKEN_URL,access_token, content_type)
        if token_status == 403:
            return jsonify({"error-code": 403, "error-desc": "header missing", "error-msg": token_validation_status}), 403
        elif token_status != True:
            logging.error(token_validation_status)
            return jsonify({"error-code":401, "error-desc": "unauthorized", "error-msg": token_validation_status}), 401

### **Validate token annotation function**

In [ ]:
def validate_token(request_method,validate_token_url):
    def decorator(function):
        @wraps(function)
        def wrapper(*args, **kwargs):
            try:
                access_token = request.headers.get(constants.HEADER_AUTHORIZATION)
                content_type = request.headers.get(constants.CONTENT_TYPE_HEADER)
                if access_token is None or access_token == "":
                    logging.error(constants.HEADER_AUTHORIZATION_MISSING)
                    return jsonify({"error-code": 403, "error-desc": "header missing", "error-msg": constants.HEADER_AUTHORIZATION_MISSING}), 403
                elif constants.BEARER_STRING not in access_token:
                    return jsonify({"error-code": 401, "error-desc": "unauthorized", "error-msg": constants.BEARER_STRING_MISSING}), 401
                if content_type is None or content_type == "":
                    logging.error(constants.HEADER_CONTENT_TYPE_MISSING)
                    return jsonify({"error-code": 403, "error-desc": "header missing", "error-msg": constants.HEADER_CONTENT_TYPE_MISSING}), 403

                response=http.request(request_method,validate_token_url,headers={constants.CONTENT_TYPE_HEADER:content_type, constants.HEADER_AUTHORIZATION:access_token})
                response_data = json.loads(response.data.decode('utf-8'))
                logging.info(response_data)
                logging.info(response.status)
                token_status = response_data['active']
                logging.info(token_status)
                # if token_status == True:
                #     return (constants.TOKEN_VALID_STATUS, token_status)
                if token_status == False:
                    return jsonify({"error-code": 401, "error-desc": "unauthorized", "error-msg": constants.TOKEN_INVALID_STATUS}), 401
                elif token_status != True:
                    return jsonify({"error-code": 401, "error-desc": "unauthorized", "error-msg": constants.TOKEN_STATUS_VALUE_ERROR}), 401
            except Exception as err:
                return jsonify(str(err)), 405
            return function(*args, **kwargs)
        return wrapper
    return decorator

### **Constants declaration**

In [ ]:
HEADER_AUTHORIZATION = "Authorization"
VALIDATE_TOKEN_URL = 'https://abc.com/dev/validatetoken'
REQUEST_METHOD = 'POST'
CONTENT_TYPE_HEADER = 'Content-Type'
CONTENT_TYPE_VALUE = 'application/json'
TOKEN_INVALID_STATUS = "Access token is invalid. Please try again with valid token."
TOKEN_VALID_STATUS = "Access token validated successfully."
TOKEN_STATUS_VALUE_ERROR = "Access token status value is invalid."
HEADER_AUTHORIZATION_MISSING = "Authorization header is missing in the request."
HEADER_CONTENT_TYPE_MISSING = "Content type header is missing in the request."
BEARER_STRING = "Bearer "
BEARER_STRING_MISSING = "Bearer keyword is missing in the access token"